In [12]:
import os
import sqlalchemy as sq

# temporary file for sql database
if os.path.exists("file.db"):
    os.remove("file.db")

In [9]:
# import prices from csv file
import pandas as pd
prices = pd.read_csv("data/price.csv", index_col=0, header=0, parse_dates=True)

In [10]:
from pyutil.sql.interfaces.symbols.symbol import Symbol, SymbolType
# Here you can go wild and point to an existing MongoDB Server. I use an inmemory-solution here...
from mongomock import MongoClient

# point to an inmemory Mongo Database
Symbol.mongo_database = MongoClient()["test"]

In [13]:
from pyutil.sql.base import Base
from pyutil.sql.session import session 


with session('sqlite:///file.db', echo=False, base=Base) as s:
    for key, data in prices.items():
        symbol = Symbol(name=key, group=SymbolType.equities, internal=key)
        # upsert the prices
        symbol.series["PX_LAST"] = data.dropna()
        s.add(symbol)
        
    # this will raise an error as a Symbol with the same name has already been defined
    # same issue if we execute this cell a second time...
    # you may need to delete file.db by hand
    # symbol = Symbol(name="A", group=SymbolType.equities, internal="Peter Maffay")
    # session.add(symbol)
    
    # commit is called whenever we leave the scope, so no need to do this explicitly here
    # s.commit()


In [16]:
with session('sqlite:///file.db', echo=False, base=Base) as s:
    for symbol in s.query(Symbol):
        print(symbol)
        print(symbol.series["PX_LAST"].tail(2))

A
2015-04-21    1202.34
2015-04-22    1200.59
Name: A, dtype: float64
B
2015-04-21    27850.49
2015-04-22    27964.84
Name: B, dtype: float64
C
2015-04-17    53954.79
2015-04-20    53761.27
Name: C, dtype: float64
D
2015-04-21    2651.41
2015-04-22    2638.95
Name: D, dtype: float64
E
2015-04-20    2100.40
2015-04-21    2097.29
Name: E, dtype: float64
F
2015-04-21    3719.38
2015-04-22    3742.77
Name: F, dtype: float64
G
2015-04-20    3059.73
2015-04-21    3065.53
Name: G, dtype: float64


In [10]:
with session('sqlite:///file.db', echo=False, base=Base) as s:
    a = s.query(Symbol).filter(Symbol.name=="A").one()
    b = s.query(Symbol).filter(Symbol.name=="B").one()
    symbols = [a,b]
    t = Symbol.frame(symbols)
    print(t)
    h = Symbol.history(symbols).tail(5)
    print(h)
    



          Name    Sector
Symbol                  
symbol(A)    A  Equities
symbol(B)    B  Equities
            symbol(A)  symbol(B)
2013-01-01    1673.78   23311.98
2013-01-02    1686.90   23311.98
2013-01-03    1663.95   23398.60
2013-01-04    1655.65   23331.09
2013-01-07    1646.95   23329.75
2013-01-08    1659.25   23111.19
2013-01-09    1657.75   23218.47
2013-01-10    1675.45   23354.31
2013-01-11    1662.80   23264.07
2013-01-14    1667.85   23413.26
2013-01-15    1679.45   23381.51
2013-01-16    1679.95   23356.99
2013-01-17    1687.55   23339.76
2013-01-18    1684.30   23601.78
2013-01-21    1690.05   23590.91
2013-01-22    1692.70   23658.99
2013-01-23    1685.85   23635.10
2013-01-24    1667.95   23598.90
2013-01-25    1658.70   23580.43
2013-01-28    1655.50   23671.88
2013-01-29    1663.80   23655.17
2013-01-30    1677.05   23822.06
2013-01-31    1663.65   23729.53
2013-02-01    1667.45   23721.84
2013-02-04    1673.70   23685.01
2013-02-05    1672.95   23148.53
2013-02-0